In [1]:
import os

In [2]:
%pwd

'c:\\Project\\Insurance-ml-Project\\research'

In [3]:
os.chdir("../")


In [4]:
%pwd

'c:\\Project\\Insurance-ml-Project'

In [28]:
from dataclasses import dataclass, field
from typing import Dict
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file_name: Path
    target_column: str
    all_params: dict


In [29]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories, save_json
import os
import pandas as pd
from src.mlProject.utils.common import read_yaml, create_directories
from src.mlProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH


In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.get("model_evaluation", {})
        params = self.params.get("ElasticNet", {})
        schema = self.schema.get("TARGET_COLUMN", {})

        root_dir = config.get("root_dir", "")
        test_data_path = config.get("test_data_path", "")
        model_path = config.get("model_path", "")
        metric_file_name = config.get("metric_file_name", "")
        target_column = schema.get("name", "")

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=root_dir,
            test_data_path=test_data_path,
            model_path=model_path,
            all_params=params,
            metric_file_name=metric_file_name,
            target_column=target_column
        )

        return model_evaluation_config


In [31]:
import os
import json
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score , accuracy_score
import joblib

In [36]:
class ModelEvaluation:
    def __init__(self, config):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = mean_squared_error(actual, pred, squared=False)
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        
        # Discretize predicted values for accuracy calculation
        threshold = 0.5  # Define a threshold to classify predictions
        pred_class = (pred > threshold).astype(int)
        actual_class = (actual > threshold).astype(int)
        acc = accuracy_score(actual_class, pred_class)

        return rmse, mae, r2, acc

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        rmse, mae, r2, acc = self.eval_metrics(test_y, predicted_qualities)

        # Create directories if they don't exist
        os.makedirs(os.path.dirname(self.config.metric_file_name), exist_ok=True)
        
        # Save metrics as JSON
        metric_results = {"RMSE": rmse, "MAE": mae, "R2": r2, "Accuracy": acc}
        with open(self.config.metric_file_name, "w") as metric_file:
            json.dump(metric_results, metric_file)


In [37]:

try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2024-02-22 05:54:39,899: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-22 05:54:39,924: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-22 05:54:40,019: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-22 05:54:40,031: INFO: common: created directory at: artifacts]
